In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
import plotly.graph_objects as go
from sklearn.model_selection import StratifiedKFold

from skimage.io import imread
import matplotlib.pyplot as plt
import cv2
import random
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor, Normalize, Compose

from tqdm import tqdm
from pathlib import Path

import torch.nn as nn
import torch.nn.functional as F
import torchvision
# import keras
%matplotlib inline
from PIL import Image
from sklearn.model_selection import train_test_split
from skimage.morphology import binary_opening, disk, label

import glob
from torchvision import datasets, models, transforms
from cv2 import imread
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, precision_score, recall_score, f1_score
import torch.optim as optim
import math 
import json

import torchmetrics 
import albumentations as A
from albumentations.pytorch import ToTensorV2
import itertools

In [ ]:
pip install https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip

In [ ]:
from torchsampler import ImbalancedDatasetSampler

In [ ]:
import wandb

! wandb login 52b3b279a9d634741e39bd8541fece124dfb8b88

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"
dataset_path = Path('../input/cassava-leaf-disease-classification')
path_df = pd.read_csv(dataset_path/'train.csv')

# BASE_DIR = "/home/linuxgpu/tsclient/97252/Desktop/Cassava_dataset"
# dataset_path = Path('/home/linuxgpu/tsclient/97252/Desktop/Cassava_dataset')
# path_df = pd.read_csv(dataset_path/'train.csv')

This competition will challenge you to distinguish between several diseases that cause material harm to the food supply of many African countries. In some cases the main remedy is to burn the infected plants to prevent further spread, which can make a rapid automated turnaround quite useful to the farmers.

In this competition we have 5 classes: 4 diseases and 1 healthy
We can find the mapping between the class number and its name in the file label_num_to_disease_map.json

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
print(json.dumps(map_classes, indent=4))

In [ ]:
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of train images: {len(input_files)}")

In [ ]:
df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
df["class_name"] = df["label"].map(map_classes)
df

In [ ]:
def show_image(class_name, examples=2, labels_df=df, train_images_path="../input/cassava-leaf-disease-classification/train_images/"):
    image_list = labels_df[labels_df['class_name'] == class_name].image_id[:examples]
    plt.figure(figsize=(20,10))
    for i, img in enumerate(image_list):
        full_path = os.path.join(train_images_path, img)
        img = Image.open(full_path)
        plt.subplot(1 ,examples, i%examples +1)
        plt.axis('off')
        plt.imshow(img)
        plt.title(class_name)

# Display classes:

### Healthy:

In [ ]:
show_image(class_name='Healthy', examples=4)

### Cassava Bacterial Blight (CBB):


In [ ]:
show_image(class_name='Cassava Bacterial Blight (CBB)', examples=4)

### Cassava Mosaic Disease (CMD):


In [ ]:
show_image(class_name='Cassava Mosaic Disease (CMD)', examples=4)

### Cassava Brown Streak Disease (CBSD)


In [ ]:
show_image(class_name='Cassava Brown Streak Disease (CBSD)', examples=4)

### Cassava Green Mottle (CGM):

In [ ]:
show_image(class_name='Cassava Green Mottle (CGM)', examples=4)

In [ ]:
df['label'].value_counts()

In [ ]:
plt.figure(figsize=(15,12))
labels = sns.barplot(df.label.value_counts().index,df.label.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

In [ ]:
source = df['label'].value_counts()
fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Label distribution')
fig.show()

In [ ]:
sfk = StratifiedKFold(10)
for train_idx, valid_idx in sfk.split(df.index, df['label']):
    df_train = df.iloc[train_idx]
    df_valid = df.iloc[valid_idx]
    break
sfk = StratifiedKFold(5)
for train_idx, valid_idx in sfk.split(df_valid.index, df_valid['label']):
    df_train = df_valid.iloc[train_idx]
    df_valid = df_valid.iloc[valid_idx]
    break
print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

In [ ]:
sfk = StratifiedKFold(5)
for train_idx, valid_idx in sfk.split(df.index, df['label']):
    df_train = df.iloc[train_idx]
    df_valid = df.iloc[valid_idx]
    break
print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

In [ ]:
CFG = {
    'img_size' : 300,
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'epochs': 10,
    'batch_size': 4,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def get_labels(self):
        return self.df['label'].tolist()
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            idx = self.df.iloc[index]['label']
            target = np.zeros(5)
            target[idx] = 1
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image'] 
        # do label smoothing
        if self.output_label == True:
            return img, target, idx
        else:
            return img

In [ ]:
def get_transform(phase: str):
    if phase == 'train':
        return A.Compose([
            A.RandomResizedCrop(height=CFG["img_size"], width=CFG["img_size"]),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.OneOf([
            A.RandomBrightnessContrast(p=0.5),A.RandomGamma(p=0.5)], p=0.5),
            A.OneOf([
            A.Blur(p=0.4),
            A.GaussianBlur(p=0.4),
            A.MotionBlur(p=0.4)], p=0.4),
            A.OneOf([
            A.GaussNoise(p=0.4),
            A.ISONoise(p=0.4),
            A.GridDropout(ratio=0.5, p=0.2),
            A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)], p=0.2),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(height=CFG["img_size"], width=CFG["img_size"]),
            A.Normalize(),
            ToTensorV2(),
        ])

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def to_numpy(tensor):
    """Auxiliary function to convert tensors into numpy arrays
    """
#     print(tensor)
#     print(type(tensor))
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEBUG = False 
def train(train_loader, model, optimizer, epoch, criterion, cv = False, model_name = None):
    m = nn.Sigmoid()
    accuracy = AverageMeter()
    acc_cls1 = AverageMeter()
    acc_cls2 = AverageMeter()
    acc_cls3 = AverageMeter()
    acc_cls4 = AverageMeter()
    acc_cls5 = AverageMeter()
    acc_list = [acc_cls1,
                acc_cls2,
                acc_cls3,
                acc_cls4,
                acc_cls5]
    losses = AverageMeter()
    model.train()
    for i, (img, target, target_num) in tqdm(enumerate(train_loader)):
            
        img = img.float()    
        img = img.to(device)
        target = target.to(device)
        target_num = target_num.to(device)
        optimizer.zero_grad()
        
        if model_name == 'inception':
            output, aux_outputs = model(img)
            loss1 = criterion(output, target)
            loss2 = criterion(aux_outputs, target)
            loss = loss1 + 0.4*loss2
        else:
            output = model(img)
            loss = criterion(output, target)

        loss.backward()

        optimizer.step()
    
        pred = m(output)        
        
        acc = torchmetrics.functional.accuracy(pred, target.int())
        accuracy_per_class = torchmetrics.functional.accuracy(pred, target.int(), average = 'none', num_classes = 5)
        target_num_unique, target_num_count = torch.unique(target_num.int(), return_counts = True)

        for i,j in zip(target_num_unique, target_num_count):
            acc_list[i.int()].update(accuracy_per_class[i.int()], j.int())

        losses.update(loss.item(), img.size(0))
        accuracy.update(acc, img.size(0))
                
        torch.cuda.empty_cache()

    if not cv:
        wandb.log({"train loss": losses.avg, "train acc": accuracy.avg*100, "train acc cls1": acc_cls1.avg*100,
                  "train acc cls2": acc_cls2.avg*100, "train acc cls3": acc_cls3.avg*100,
                   "train acc cls4": acc_cls4.avg*100, "train acc cls5": acc_cls5.avg*100}, step = epoch)
    return losses.avg, accuracy.avg*100

In [ ]:
best_acc = 0
def validate(dataloader, model, criterion, optimizer, epoch, cv = False):
    m = nn.Sigmoid()
    acc = AverageMeter()
    acc_cls1 = AverageMeter()
    acc_cls2 = AverageMeter()
    acc_cls3 = AverageMeter()
    acc_cls4 = AverageMeter()
    acc_cls5 = AverageMeter()
    acc_list = [acc_cls1,
                acc_cls2,
                acc_cls3,
                acc_cls4,
                acc_cls5]
    losses = AverageMeter()
    model.eval()
    loss_accumlated = 0
    for i, (img, target, target_num) in tqdm(enumerate(dataloader)):
        
        img = img.float()
        img = img.to(device)
        target = target.to(device)
        target_num = target_num.to(device)
        
        optimizer.zero_grad() 
        
        output = model(img) 
        loss = criterion(output,target)
        losses.update(loss.item(), img.size(0))
        loss_accumlated += loss.item()
                
        pred = m(output)
                
        accuracy = torchmetrics.functional.accuracy(pred,  target.int())
        accuracy_per_class = torchmetrics.functional.accuracy(pred, target.int(), average = 'none', num_classes = 5)
        acc.update(accuracy, img.size(0))
        
        target_num_unique, target_num_count = torch.unique(target_num.int(), return_counts = True)

        for i,j in zip(target_num_unique, target_num_count):
            acc_list[i.int()].update(accuracy_per_class[i.int()], j.int())
            
        torch.cuda.empty_cache()


    if not cv:
        wandb.log({"val loss": losses.avg, "val acc": acc.avg*100, "val acc cls1": acc_cls1.avg*100,
                   "val acc cls2": acc_cls2.avg*100, "val acc cls3": acc_cls3.avg*100,
                   "val acc cls4": acc_cls4.avg*100, "val acc cls5": acc_cls5.avg*100}, step = epoch)
    return losses.avg, acc.avg*100

In [ ]:
def create_data():
    BASE_DIR = "../input/cassava-leaf-disease-classification/"
    dataset_path = Path('../input/cassava-leaf-disease-classification')
    path_df = pd.read_csv(dataset_path/'train.csv')

    with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
        map_classes = json.loads(file.read())
        map_classes = {int(k) : v for k, v in map_classes.items()}
    df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
    df["class_name"] = df["label"].map(map_classes)
    return df

In [ ]:
def create_loaders(df_train, df_valid, batch_size, balanced = False):

    train_dataset = CassavaDataset(df_train, '../input/cassava-leaf-disease-classification/train_images', transforms=get_transform("train"))
    if balanced:
        train_loader = DataLoader(
                        train_dataset,
                        sampler=ImbalancedDatasetSampler(train_dataset),
                        batch_size= batch_size, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)
    else:
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

    val_dataset = CassavaDataset(df_valid, '../input/cassava-leaf-disease-classification/train_images', transforms=get_transform(None))
    valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return train_loader, valid_loader

In [ ]:
def set_parameter_requires_grad(model, step_wise):
    if step_wise:
        for param in model.parameters():
            param.requires_grad = False
    else:
        params_len = len(list(model.parameters()))
        counter = 0
        for param in model.parameters():
            if counter >= params_len/2:
                break
            param.requires_grad = False
            counter +=1

In [ ]:
def create_model(model_conf, step_wise):
    if model_conf == "resnet50":
        model = models.resnet50(pretrained = True)
        set_parameter_requires_grad(model, step_wise)
        in_features = model.fc.in_features
        model.fc = torch.nn.Linear(in_features, 5)
    elif model_conf == "vgg16":
        model = models.vgg16_bn(pretrained=True)
        set_parameter_requires_grad(model, step_wise)
        in_features = model.classifier[6].in_features
        model.classifier[6] = torch.nn.Linear(in_features, 5)
    elif model_conf == "inception":
        model = models.inception_v3(pretrained=True)
        set_parameter_requires_grad(model, step_wise)
        # Handle the auxilary net
        num_ftrs = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Linear(num_ftrs, 5)
        # Handle the primary net
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs,5)
    elif model_conf == "ResNext50":
        model = models.resnext50_32x4d(pretrained = True)
        set_parameter_requires_grad(model, step_wise)
        in_features = model.fc.in_features
        model.fc = torch.nn.Linear(in_features, 5)
    elif model_conf == "mobileNet":   
        model_mobilenet = models.mobilenet_v3_large(pretrained = True)
        in_features = model_mobilenet.classifier[3].in_features
        model_mobilenet.classifier[3] = torch.nn.Linear(in_features, 5)

    model = model.cuda()
    return model

In [ ]:
def create_opt(network, optimizer, learning_rate, weight_decay = 0, momentum = 0.9):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                            lr=learning_rate, momentum = momentum, weight_decay = weight_decay)
    elif optimizer == "adamw":
        optimizer = optim.AdamW(network.parameters(),
                               lr=learning_rate, weight_decay = weight_decay)
    return optimizer
    

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, logist=False, reduce='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha # the scalar factor between 0 and 1 
        self.gamma = gamma # focusing parameter(always positive) that reduces the relative loss for well-classified examples and puts more focus on hard misclassified examples
        self.BCE_Logits_loss = nn.BCEWithLogitsLoss()
        #self.logist = logist # log probabilities 
        self.reduce = reduce # Specifies the reduction to apply to the output - none/mean/sum. ‘none’: no reduction will be applied, ‘mean’: the sum of the output will be divided by the number of elements in the output, ‘sum’: the output will be summed.

    def forward(self, inputs, targets):
        BCE_loss = self.BCE_Logits_loss(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [ ]:
def create_criterion(loss):
    if loss == "BCE":
        criterion = nn.BCEWithLogitsLoss()
    if loss == "FL":
        criterion = FocalLoss()
    return criterion

In [ ]:
def unfreeze(model, step):
    params_len = len(list(model.parameters()))
    counter = 0
    for param in model.parameters():
        if counter >= (step/3)*params_len:
            param.requires_grad = True
        counter +=1

In [ ]:
def train_hyperparameters(config=None):
    with wandb.init(config=config):
        config = wandb.config
        EPOCHS = config.epochs
        df = create_data()
#         sfk = StratifiedKFold(5)
#         for train_idx, valid_idx in sfk.split(df.index, df['label']):
#             df_train = df.iloc[train_idx]
#             df_val = df.iloc[valid_idx]
#             break
        sfk = StratifiedKFold(5)
        for train_idx, valid_idx in sfk.split(df.index, df['label']):
            df_train = df.iloc[train_idx]
            df_valid = df.iloc[valid_idx]
            break
        loader_train, loader_val = create_loaders(df_train, df_valid, config.batch_size, config.balance == 'true')
        model = create_model(config.model, config.step_wise)
        opt = create_opt(model, config.optimizer, config.learning_rate, config.weight_decay, config.momentum)
        criterion = create_criterion(config.criterion)
        losses_train = []
        acc_train = []
        losses_val = []
        acc_val = []
        step_wise_count = 2
        if config.scheduler:
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.2, patience=5, eps=1e-6)

        for e in range(EPOCHS):
            result_train = train(loader_train, model, opt, e, criterion, config.cv == 'true', config.model)
            result_val = validate(loader_val, model, criterion, opt, e, config.cv == 'true')
            if config.step_wise and e % 5 == 0:
                unfreeze(model, step_wise_count)
                step_wise_count -= 1
            if config.scheduler:
                scheduler.step(result_val[0])

In [ ]:
import math

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'val acc',
    'goal': 'maximize'  
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'value': 'adamw'
        },
    'model': {
        'value': 'resnet50'
        },
    'cv': {
        'value' : 'true'
    },
    'weight_decay':{
        'value' : 0.006764
    },
    'momentum':{
        'value' : 0.9
    },
    'criterion':{
        'value' : "FL"
    },
    'balance':{
        'value' : 'false'
    },
    'step_wise':{
        'value' : True
    }
    }

sweep_config['parameters'] = parameters_dict


parameters_dict.update({
    'epochs': {
        'value': 20}
    })

parameters_dict.update({
    'learning_rate': {
        'value' : 0.0000881
      },
    'scheduler': {
        'value' : True
      },
    'batch_size': {
        # with evenly-distributed logarithms 
        'value': 32
      }
    })

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="ML_seminar", entity="seminar")

In [ ]:
# wandb.agent(sweep_id, train_hyperparameters, count=1)

In [ ]:
def train_cv(config=None):
    with wandb.init(config=config):
        config = wandb.config
        EPOCHS = config.epochs
        df = create_data()
        all_losses_train = []
        all_acc_train = []
        all_losses_val = []
        all_acc_val = []
        max_val_acc = 0
#         sfk = StratifiedKFold(14)
#         for train_idx, valid_idx in sfk.split(df.index, df['label']):
#             df_train = df.iloc[train_idx]
#             df_val = df.iloc[valid_idx]
#             break
        sfk = StratifiedKFold(5)
        for train_idx, valid_idx in sfk.split(df.index, df['label']):
            df_train = df.iloc[train_idx]
            df_valid = df.iloc[valid_idx]
            
            loader_train, loader_val = create_loaders(df_train, df_valid, config.batch_size, config.balance == 'true')
            model = create_model(config.model, config.step_wise)
            opt = create_opt(model, config.optimizer, config.learning_rate)
            criterion = create_criterion(config.criterion)
            losses_train = []
            acc_train = []
            losses_val = []
            acc_val = []
            if config.scheduler:
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.2, patience=5, eps=1e-6)
            step_wise_count = 2
            for e in range(EPOCHS):
                result_train = train(loader_train, model, opt, e, criterion, config.cv == 'true', config.model)
                result_val = validate(loader_val, model, criterion, opt, e, config.cv == 'true')
                
                if config.step_wise and e % 5 == 0:
                    unfreeze(model, step_wise_count)
                    step_wise_count -= 1
                
                if config.scheduler:
                    scheduler.step(result_val[0])
                    
                losses_train.append(result_train[0])
                acc_train.append(result_train[1])
                losses_val.append(result_val[0])
                acc_val.append(result_val[1])
                
                if max_val_acc < result_val[1]:
                    max_val_acc = result_val[1]
                    torch.save(model, './model_resnet50')
            
            all_losses_train.append(losses_train)
            all_acc_train.append(acc_train)
            all_losses_val.append(losses_val)
            all_acc_val.append(acc_val)
            
        total_losses_train = []
        total_acc_train = []
        total_losses_val = []
        total_acc_val = []
        
        for i in range(EPOCHS):
            temp1 = 0
            temp2 = 0
            temp3 = 0
            temp4 = 0
            
            for j in range(5):
                temp1 = temp1 + all_losses_train[j][i]
                temp2 = temp2 + all_acc_train[j][i]
                temp3 = temp3 + all_losses_val[j][i]
                temp4 = temp4 + all_acc_val[j][i]
            
            total_losses_train.append(temp1/5)
            total_acc_train.append(temp2/5)
            total_losses_val.append(temp3/5)
            total_acc_val.append(temp4/5)
            
        for i in range(EPOCHS):
            wandb.log({"train loss cv": total_losses_train[i], "train acc cv": total_acc_train[i], "val loss cv": total_losses_val[i], "val acc cv": total_acc_val[i]})

            # return losses_train, acc_train, losses_val, acc_val

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="ML_seminar", entity="seminar")

In [ ]:
# wandb.agent(sweep_id, train_cv, count=1)

In [ ]:
df = create_data()
sfk = StratifiedKFold(5)
for train_idx, valid_idx in sfk.split(df.index, df['label']):
    df_train = df.iloc[train_idx]
    df_valid = df.iloc[valid_idx]
    break
val_dataset = CassavaDataset(df_valid, '../input/cassava-leaf-disease-classification/train_images', transforms=get_transform(None))
valid_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def Sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
def predict(model, loader):
    y_true = np.empty(shape=(0, 5), dtype=np.int)
    y_pred_proba = np.empty(shape=(0, 5), dtype=np.int)
    model = model.to(DEVICE)
    model.eval()
    stream = tqdm(loader)
    pred_avg = [0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0]
    pred_avg_0 = [0, 0, 0, 0, 0]
    pred_counts_0 = [0, 0, 0, 0, 0]
    
    for batch, (X, y, y_n) in enumerate(stream, start=1):
        X = X.to(DEVICE)
        y = to_numpy(y.to(DEVICE))
        pred = to_numpy(model(X))
#         pred = Sigmoid(pred)
        
        pred_max = list(np.argmax(pred, 1))
        
        for i in range(len(pred_max)):
            pred[i] = [0,0,0,0,0]
            pred[i][pred_max[i]] = 1
        
        for i,j in zip(pred, y):
             for k in range(5):
                    if j[k] == 1:
                        pred_avg[k] += i[k]
                        pred_counts[k] += 1
                    else:
                        pred_avg_0[k] += i[k]
                        pred_counts_0[k] += 1
        
        
        y_true = np.vstack((y_true, y))
        y_pred_proba = np.vstack((y_pred_proba, pred))
    avg = [i/j for i, j in zip (pred_avg,pred_counts)]
    avg_0 = [i/j for i, j in zip (pred_avg_0,pred_counts_0)]
    print('average output values for true label' + str(avg))
    print('average output values for false label' + str(avg_0))
    return y_true, y_pred_proba

In [ ]:
classes = [
        '0', 
        '1', 
        '2', 
        '3', 
        '4']

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix


def plot_confusion_matrix(
    y_test, 
    y_pred_proba, 
    label_names=classes
)-> None:
    """
    """
    y_pred = y_pred_proba
    c_matrices = multilabel_confusion_matrix(y_test, y_pred)
    
    cmap = plt.get_cmap('Blues')
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))

    for cm, label, ax in zip(c_matrices, label_names, axes.flatten()):
        sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap=cmap);

        ax.set_xlabel('Predicted labels');
        ax.set_ylabel('True labels'); 
        ax.set_title(f'{label}');

    plt.tight_layout()    
    plt.show()

In [ ]:
# model = torch.load('../input/inception/model_inception')
# y_true, y_pred_proba = predict(model, valid_loader)

In [ ]:
import torch.nn.functional as F

class MyEnsemble(nn.Module):
    def __init__(self, models):
        super(MyEnsemble, self).__init__()
        self.models = []
        for model in models:
            self.models.append(model)
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        allX = []
        for model in self.models:
            allX.append(self.Sigmoid(model(x.clone())))
        x = sum(allX)/len(allX)
        return x

In [ ]:
model_inception = torch.load('../input/inception/model_inception')
model_resnet50 = torch.load('../input/resnet/model_resnet50')
model_vgg16 = torch.load('../input/vgg16/model_vgg16')

In [ ]:
model = MyEnsemble([model_resnet50, model_inception, model_vgg16])
model = model.cuda()

In [ ]:
torch.save(model, "./ensemble")

In [ ]:
y_true, y_pred_proba = predict(model, valid_loader)

In [ ]:
plot_confusion_matrix(y_true, y_pred_proba)

In [ ]:
from sklearn.metrics import confusion_matrix

def get_conf_mat(model, loader, num_classes):
    nb_classes = num_classes
    # Initialize the prediction and label lists(tensors)
    predlist=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

    with torch.no_grad():
        for i, (inputs, classes, _) in enumerate(loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            
            
            _, preds = torch.max(outputs, 1)
            _, classes = torch.max(classes, 1)
            # Append batch prediction results

            predlist=torch.cat([predlist,preds.view(-1).cpu()])
            lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

    # Confusion matrix
    return confusion_matrix(lbllist.numpy(), predlist.numpy()), lbllist.numpy(), predlist.numpy()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """

    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.

    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
plt.figure(figsize=(20,15), dpi=50)
plt.rcParams.update({'font.size': 18})
conf_mat, y_true, y_pred = get_conf_mat(model, valid_loader, len(classes))
plot_confusion_matrix(conf_mat, classes=classes, title='Confusion matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))